In [9]:
from catboost import Pool
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import joblib
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv('../Database/train_modified2.csv', index_col='ID')
X = train.drop(columns=['대출등급'])
y = train['대출등급']

In [4]:
X

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출기간,근로기간,주택소유상태,대출목적
ID,,,,,,,,,,,,,
TRAIN_00000,-0.563864,-0.220228,-0.039061,-0.852450,-0.376088,-0.800307,-0.972790,-0.038438,-0.072596,0,9,3,1
TRAIN_00001,-0.377979,0.370322,0.317312,-0.356112,-0.376088,-0.436819,-0.441089,-0.038438,-0.072596,1,2,1,10
TRAIN_00002,-0.610335,0.020813,-1.109217,-0.935172,-0.376088,0.103269,-0.627628,-0.038438,-0.072596,0,8,1,1
TRAIN_00003,-0.377979,0.382374,-0.434915,-0.852450,-0.376088,-0.483278,-0.624984,-0.038438,-0.072596,0,11,1,1
TRAIN_00004,-0.029446,-0.222879,0.635242,-0.521558,-0.376088,-0.577936,-0.634415,-0.038438,-0.072596,1,15,3,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_96289,-0.377979,1.165756,-1.033371,0.636562,-0.376088,0.147965,0.145240,-0.038438,-0.072596,0,2,1,3
TRAIN_96290,1.016156,0.382374,-1.466629,-0.025221,-0.376088,-0.232336,0.969657,-0.038438,-0.072596,1,2,1,10
TRAIN_96291,-0.377979,-0.099707,-0.834925,-0.273389,-0.376088,0.648623,-0.424788,-0.038438,-0.072596,0,0,1,3


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cat_features=[i for i in range(9,13)]

train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

In [8]:
cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth = 10,
    l2_leaf_reg=3,
    task_type = 'CPU',
    loss_function = 'MultiClassOneVsAll',
    boosting_type = 'Ordered'
)
print(type(cat_model))
cat_model.fit(train_pool, eval_set=val_pool, verbose=5)

joblib.dump(cat_model, '../Files/cat_model.pkl')

<class 'catboost.core.CatBoostClassifier'>
0:	learn: 0.6382292	test: 0.6382342	best: 0.6382342 (0)	total: 1.23s	remaining: 20m 25s
5:	learn: 0.4631596	test: 0.4631416	best: 0.4631416 (5)	total: 14.2s	remaining: 39m 13s
10:	learn: 0.3760861	test: 0.3762557	best: 0.3762557 (10)	total: 22.7s	remaining: 33m 57s
15:	learn: 0.3192030	test: 0.3195999	best: 0.3195999 (15)	total: 38.5s	remaining: 39m 29s
20:	learn: 0.2824359	test: 0.2829399	best: 0.2829399 (20)	total: 58s	remaining: 45m 5s
25:	learn: 0.2588964	test: 0.2595418	best: 0.2595418 (25)	total: 1m 9s	remaining: 43m 28s
30:	learn: 0.2404353	test: 0.2413732	best: 0.2413732 (30)	total: 1m 22s	remaining: 43m
35:	learn: 0.2263191	test: 0.2275975	best: 0.2275975 (35)	total: 1m 36s	remaining: 43m
40:	learn: 0.2165392	test: 0.2181991	best: 0.2181991 (40)	total: 1m 52s	remaining: 43m 44s
45:	learn: 0.2073290	test: 0.2092435	best: 0.2092435 (45)	total: 2m 11s	remaining: 45m 34s
50:	learn: 0.2000469	test: 0.2021733	best: 0.2021733 (50)	total: 2m 

NameError: name 'joblib' is not defined

In [10]:
joblib.dump(cat_model, '../Files/cat_model.pkl')

['../Files/cat_model.pkl']

In [11]:
print("CatBoost Accuracy:", accuracy_score(y_test, cat_model.predict(X_test)))

CatBoost Accuracy: 0.8555999792304897


In [63]:
cat_model.save_model('../Files/cat_model.bin')

In [64]:
loaded_model = CatBoostClassifier()
loaded_model.load_model('../Files/cat_model.bin')
loaded_model.fit(train_pool, eval_set=val_pool, verbose=2)

0:	learn: 0.6366766	test: 0.6366857	best: 0.6366857 (0)	total: 5.5s	remaining: 1h 31m 38s
2:	learn: 0.5483135	test: 0.5483425	best: 0.5483425 (2)	total: 7.65s	remaining: 42m 22s
4:	learn: 0.4873472	test: 0.4873972	best: 0.4873972 (4)	total: 9.88s	remaining: 32m 46s
6:	learn: 0.4358618	test: 0.4359404	best: 0.4359404 (6)	total: 13.4s	remaining: 31m 36s
8:	learn: 0.3951242	test: 0.3952551	best: 0.3952551 (8)	total: 16.9s	remaining: 30m 58s
10:	learn: 0.3655918	test: 0.3658047	best: 0.3658047 (10)	total: 20.2s	remaining: 30m 15s
12:	learn: 0.3404294	test: 0.3407176	best: 0.3407176 (12)	total: 23.6s	remaining: 29m 55s
14:	learn: 0.3191448	test: 0.3195833	best: 0.3195833 (14)	total: 27.3s	remaining: 29m 52s
16:	learn: 0.2999630	test: 0.3004349	best: 0.3004349 (16)	total: 30.9s	remaining: 29m 43s
18:	learn: 0.2857266	test: 0.2863691	best: 0.2863691 (18)	total: 34.3s	remaining: 29m 30s


KeyboardInterrupt: 

In [ ]:
cat_model = CatBoostClassifier(random_state=2024,
                               n_estimators=1000,
                               learning_rate=0.01,
                               depth=15,
                               l2_leaf_reg=3,
                               metric_period=1000,
                               task_type='GPU')



cat_model.fit(X_train, y_train, cat_features=cat_features)
print("CatBoost Accuracy:", accuracy_score(y_test, cat_model.predict(X_test)))